In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

from pynba import possessions_from_file, bokeh_theme
from pynba.rapm import TimeDecayedRAPM, _RAPMData

In [2]:
league = "nba"
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
years = [2016]
season_type = "Regular Season"

possessions = pd.concat([possessions_from_file(league, year, season_type) for year in years])

In [3]:
rapm_data = _RAPMData.from_possessions(possessions)

In [4]:
half_life = 1.375
off_prior = 3.1875
def_prior = 2.375
date = rapm_data.time.max()

time_decayed_rapm = TimeDecayedRAPM.from_data(rapm_data, date, half_life, off_prior, def_prior)

In [32]:
time_decayed_rapm.stats[-50:]

,name,rapm,off_rapm,def_rapm,rapm_std,off_rapm_std,def_rapm_std,off_poss,def_poss,raw_pm,off_raw_pm,def_raw_pm
1626148,Anthony Brown,1.350815,-0.288930,1.639745,1.816917,1.496604,1.028633,153.979437,155.135430,-2.819280,-4.694574,1.875294
1626149,Montrezl Harrell,1.450758,0.930074,0.520685,1.760908,1.473923,0.961997,99.643264,99.506072,5.019474,2.928346,2.091128
1626151,Aaron Harrison,-0.996708,-1.252160,0.255451,1.106045,0.949755,0.565964,26.530659,26.136754,-6.661630,-12.586758,5.925128
1626153,Delon Wright,0.700327,0.161662,0.538666,1.473044,1.245528,0.784430,61.896509,61.620242,6.668214,0.804128,5.864085
1626154,RJ Hunter,-1.419554,-2.404497,0.984943,1.647970,1.386422,0.889209,81.455529,81.579162,-3.642826,-12.470682,8.827856
1626155,Sam Dekker,0.051138,0.154824,-0.103686,0.350683,0.301709,0.178758,1.739083,1.970639,14.868900,23.776361,-8.907461
1626156,D'Angelo Russell,-2.732464,-2.303878,-0.428586,1.739251,1.384529,1.050636,586.541679,587.889601,-11.702831,-6.263686,-5.439145
1626157,Karl-Anthony Towns,0.880961,2.489558,-1.608597,1.634768,1.333357,0.946185,682.458904,684.828885,-2.665847,2.093421,-4.759268
1626158,Richaun Holmes,1.386107,1.408629,-0.022522,1.871384,1.534330,1.069305,184.606875,183.605285,-3.416249,-1.720607,-1.695642
1626159,Justise Winslow,0.697899,-1.629829,2.327728,1.805107,1.434853,1.093145,570.829559,567.342257,2.774115,-1.448418,4.222532


In [5]:
model = time_decayed_rapm.model

In [29]:
k_hat_cov

array([[ 1.79192159e+00, -1.26182380e-03, -1.28748576e-02, ...,
         2.63336777e-03, -3.11501346e-04, -2.23817957e-03],
       [-1.26182380e-03,  1.78695837e+00, -1.04609646e-04, ...,
         1.75420073e-03, -1.07987866e-03, -8.09222107e-05],
       [-1.28748576e-02, -1.04609646e-04,  1.95332616e+00, ...,
         4.66523677e-03, -8.98287509e-04, -1.26441621e-03],
       ...,
       [ 2.63336777e-03,  1.75420073e-03,  4.66523677e-03, ...,
         9.84950817e-01, -1.28326089e-03, -9.77857135e-04],
       [-3.11501346e-04, -1.07987866e-03, -8.98287509e-04, ...,
        -1.28326089e-03,  2.59599328e-01,  6.11296414e-02],
       [-2.23817957e-03, -8.09222107e-05, -1.26441621e-03, ...,
        -9.77857135e-04,  6.11296414e-02,  4.90470712e-01]])

In [28]:
A_inv = np.linalg.inv(model.A.toarray())
XT_W_X = model.XT_W.multiply(rapm_data.y_var).dot(model.XT_W.T).toarray()
k_hat_cov = A_inv.dot(XT_W_X).dot(A_inv.T)
# A_inv[A_inv < thresh] = 0
# A_inv = sparse.csr_matrix(A_inv)
# H = model.X.dot(A_inv.dot(model.XT_W))  # this is the "hat" or projection matrix
# M = sparse.eye(H.shape[0]) - H  # this is the annihilator matrix
# degrees_of_freedom = M.T.dot(M).trace()
# residuals = model.y - model.predict(model.X)
# est_err_var = (residuals * model.w.reshape(-1)).dot(residuals) / degrees_of_freedom
# k_hat_cov = est_err_var * A_inv

In [15]:
from pynba.rapm import SparseWeightedRidgeRegression

In [23]:
SparseWeightedRidgeRegression.k_hat_cov = k_hat_cov

In [11]:
setattr(model, "k_hat_cov", k_hat_cov)

FrozenInstanceError: cannot assign to field 'k_hat_cov'

In [8]:
model.k_hat_cov = k_hat_cov

FrozenInstanceError: cannot assign to field 'k_hat_cov'

In [13]:
H.nnz / (H.shape[0] * H.shape[1]) * 1e2

9.160343541330516

In [9]:
H.data

array([0.000489  , 0.00011537, 0.00042792, ..., 0.00184676, 0.00184676,
       0.00699794])

In [10]:
H.data.max()

0.1598749687488612

In [13]:
dom = np.logspace(-6, 0,7)
temp = [(H.data > val).sum() for val in dom]


In [23]:
np.percentile(np.abs(sparse.linalg.inv(model.A.tocsc()).toarray()), np.arange(0, 100, 11))

array([4.22217613e-09, 4.04435966e-04, 1.24894790e-03, 2.51346245e-03,
       4.01149867e-03, 5.65874778e-03, 7.56284710e-03, 1.01201270e-02,
       1.54849845e-02, 1.35228163e-01])

In [21]:
np.percentile(H.data, np.arange(0, 100, 11))

array([6.20506709e-06, 3.73561222e-05, 1.32018412e-04, 4.56195649e-04,
       7.48325078e-04, 1.09813044e-03, 1.59550654e-03, 2.34093064e-03,
       3.72957460e-03, 1.63649311e-02])

In [8]:
np.logspace(H.data.min(), H.data.max()

6.205067089296408e-06

In [22]:
fig = figure()


373579633

In [ ]:
sns.displot(H.data)

In [24]:
H.data

array([0.00265216, 0.00889983, 0.0018699 , ..., 0.00205244, 0.01313984,
       0.00653891])

In [ ]:
k_hat_cov = time_decayed_rapm.model.k_hat_cov

In [ ]:
x_var = k_hat_cov.diagonal()
x_var

In [ ]:

off_def_cov = k_hat_cov.diagonal(time_decayed_rapm.data.n_players)

In [ ]:
from scipy import sparse

In [ ]:
time_decayed_rapm.model.w

In [ ]:
thing = (time_decayed_rapm.model.XT_W > 0).toarray()

In [ ]:
rapm_data.X.shape

In [ ]:
time_decayed_rapm.model.X.shape[0]

In [ ]:
thing.any(1).sum()

In [ ]:
residuals = time_decayed_rapm.model.y - time_decayed_rapm.model.predict(time_decayed_rapm.model.X)
degrees_of_freedom = time_decayed_rapm.model.X.shape[0] - time_decayed_rapm.model.X.shape[1]
est_err_var = (residuals * time_decayed_rapm.model.w.reshape(-1)).dot(residuals) / degrees_of_freedom
k_hat_cov = est_err_var * sparse.linalg.inv(time_decayed_rapm.model.A.tocsc())

In [ ]:
k_hat_cov = time_decayed_rapm.model.k_hat_cov
n_players = time_decayed_rapm.data.n_players

x_var = k_hat_cov.diagonal()
off_def_cov = k_hat_cov.diagonal(n_players)
off_rapm_var, def_rapm_var = x_var[:n_players], x_var[n_players:]
rapm_var = off_rapm_var + def_rapm_var + 2 * off_def_cov
rapm_std, off_rapm_std, def_rapm_std = np.sqrt(rapm_var), np.sqrt(off_rapm_var), np.sqrt(def_rapm_var)

In [ ]:
(time_decayed_rapm.model.w > 0).all()

In [ ]:
stats[-1]

In [ ]:
stats[0].isna().any()

In [ ]:
date

In [ ]:
half_life = 1.375
off_prior = 3.1875
def_prior = 2.375

dates = np.unique(rapm_data.time)
stats_dict = {}

for date in dates:
    sub_rapm_data = rapm_data[rapm_data.time <= date]
    time_decayed_rapm = TimeDecayedRAPM.from_data(sub_rapm_data, date, half_life, off_prior, def_prior)
    stats_dict[date] = time_decayed_rapm.stats